### Question Answer Application
The goal of Question Answering is to find the answer to a question given a question and an accompanying context. The predicted answer will be either a span of text from the context or an empty string (indicating the question cannot be answered from the context).

In [1]:
!pip install simpletransformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.7/250.7 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 59.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 29.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 102.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 50.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 68.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.5/199.5 kB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 2

In [2]:
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import json,ast

In [4]:
f = open('bert_dataset.txt','r').read()
data = ast.literal_eval(f)

In [5]:
def modify(con):
  id = con['id']
  imp = con['is_impossible']
  que = con['question']
  qa = con['qas'][0]
  con.pop('qas')
  con.pop('id')
  con.pop('is_impossible')
  con.pop('question')
  con['qas'] = [{'id':id,'answers':[{'answer_start': qa['answer_starts'],'text':qa['text']}],'is_impossible':imp,'question':que}]
  

In [6]:
for d in data:
  modify(d)

In [7]:
import random
import numpy as np
data_np = np.array(data)

In [8]:
from os import initgroups
indices = np.random.permutation(data_np.shape[0])
train , test = data_np[indices[:620]] , data_np[indices[620:]]

In [9]:
train , test = train.tolist() , test.tolist()

In [10]:
import logging

from simpletransformers.question_answering import QuestionAnsweringModel, QuestionAnsweringArgs

In [22]:
model_type="xlnet"
model_name= "bert-base-cased"
if model_type == "bert":
    model_name = "bert-base-cased"

elif model_type == "roberta":
    model_name = "roberta-base"

elif model_type == "distilbert":
    model_name = "distilbert-base-cased"

elif model_type == "distilroberta":
    model_type = "roberta"
    model_name = "distilroberta-base"

elif model_type == "electra-base":
    model_type = "electra"
    model_name = "google/electra-base-discriminator"

elif model_type == "electra-small":
    model_type = "electra"
    model_name = "google/electra-small-discriminator"

elif model_type == "xlnet":
    model_name = "xlnet-base-cased"

In [23]:
# Configure the model 
model_args = QuestionAnsweringArgs()
model_args.train_batch_size = 16
model_args.evaluate_during_training = True
model_args.n_best_size=3
model_args.num_train_epochs=5


In [24]:
### Advanced Methodology
train_args = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "use_cached_eval_features": True,
    "output_dir": f"outputs/{model_type}",
    "best_model_dir": f"outputs/{model_type}/best_model",
    "evaluate_during_training": True,
    "max_seq_length": 128,
    "num_train_epochs": 5,
    "evaluate_during_training_steps": 1000,
    #"wandb_project": "Question Answer Application",
    #"wandb_kwargs": {"name": model_name},
    "save_model_every_epoch": False,
    "save_eval_checkpoints": False,
    "n_best_size":3,
    # "use_early_stopping": True,
    # "early_stopping_metric": "mcc",
    # "n_gpu": 2,
    # "manual_seed": 4,
    # "use_multiprocessing": False,
    "train_batch_size": 128,
    "eval_batch_size": 64,
    # "config": {
    #     "output_hidden_states": True
    # }
}

In [25]:
train

[{'context': 'end x x Add  to the right side of the deque.. appendleft x x Add  to the left side of the deque.. clear Remove all elements from the deque leaving it ',
  'qas': [{'id': 145,
    'answers': [{'answer_start': 73, 'text': 'left'}],
    'is_impossible': False,
    'question': 'What does insert i x x insert into the deque at position?'}]},
 {'context': 'rderedDict dict subclass that remembers the order entries were added. defaultdict dict subclass that calls a factory function to supply missing values',
  'qas': [{'id': 128,
    'answers': [{'answer_start': 70, 'text': 'defaultdict'}],
    'is_impossible': False,
    'question': 'What dict subclass calls a factory function to supply missing values?'}]},
 {'context': '  and then by the order encountered in the right operand.. . . . . . . elements Return an iterator over elements repeating each as many times as its  ',
  'qas': [{'id': 141,
    'answers': [{'answer_start': 73, 'text': 'left'}],
    'is_impossible': False,
    '

In [26]:
model = QuestionAnsweringModel(
    model_type,model_name, args=train_args
)

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForQuestionAnswering: ['lm_loss.bias', 'lm_loss.weight']
- This IS expected if you are initializing XLNetForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForQuestionAnswering were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['end_logits.dense_1.weight', 'answer_class.dense_0.weight', 'end_logits.dense_0.bias', 'answer_class.dense_0.bias', 'start_logits.dense.bias', 'end_logits.dense_1.bias', 'end_logits.dense_0.weight', 'start_logits.dense.weight',

In [27]:
### Remove output folder
!rm -rf outputs

In [28]:
# Train the model
model.train_model(train, eval_data=test)

convert squad examples to features:   0%|          | 0/620 [00:00<?, ?it/s]Could not find answer: 'elements' vs. 'left'
Could not find answer: 'typename and field_names)' vs. 'keyword-only arguments'
Could not find answer: ', an IndexError is raised.' vs. 'grow to an arbitrary length'
Could not find answer: 'c.total() 15 New' vs. 'elements'
Could not find answer: 'are' vs. '.'
Could not find answer: 'many' vs. 'the'
Could not find answer: 'mappings' vs. 'reference'
Could not find answer: ''i', 'j',' vs. 'G H I'
Could not find answer: 'dictionaries, return None as a default' vs. 'propagated unchanged'
Could not find answer: 'a zero' vs. 'zero 0'
Could not find answer: 'default_factory' vs. 'dict'
Could not find answer: 'Cache the second' vs. '[81, 64, 49]'
Could not find answer: 'underlying' vs. 'a key'
Could not find answer: 'baseline))' vs. 'first'
Could not find answer: 'typename and field_names)' vs. 'keyword-only arguments'
Could not find answer: 'required argument,' vs. 'version 3

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Running Epoch 0 of 5:   0%|          | 0/3 [00:00<?, ?it/s]

/usr/local/lib/python3.9/dist-packages/torch/optim/lr_scheduler.py:139: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "

convert squad examples to features: 100%|██████████| 69/69 [00:00<00:00, 496.11it/s]

add example index and unique id: 100%|██████████| 69/69 [00:00<00:00, 271488.72it/s]


Running Evaluation:   0%|          | 0/2 [00:00<?, ?it/s]

...

Running Epoch 3 of 5:   0%|          | 0/3 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/2 [00:00<?, ?it/s]

Running Epoch 4 of 5:   0%|          | 0/3 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/2 [00:00<?, ?it/s]

(15,
 {'global_step': [3, 6, 9, 12, 15],
  'correct': [0, 0, 0, 0, 0],
  'similar': [21, 21, 21, 21, 21],
  'incorrect': [4, 4, 4, 4, 4],
  'train_loss': [9.041668030536126e+29,
   8.958335262632815e+29,
   8.958336018211452e+29,
   8.791668971247556e+29,
   8.791669726826193e+29],
  'eval_loss': [0.05257009156048298,
   0.05257009156048298,
   0.07513132691383362,
   0.03290613926947117,
   0.031262872740626335]})

In [29]:
test

[{'context': ' If the insertion would cause a bounded deque to grow beyond ,  an IndexError is raised. New in version 3.5.. pop Remove and return an element from th',
  'qas': [{'id': 146,
    'answers': [{'answer_start': 64, 'text': 'an IndexError is raised'}],
    'is_impossible': False,
    'question': 'What happens if the insertion causes a bounded deque to be inserted?'}]},
 {'context': 'n to supporting the methods and operations of mutable sequences,  UserList instances provide the following attribute:. data. A real list object used t',
  'qas': [{'id': 128,
    'answers': [{'answer_start': 66, 'text': 'UserList instances'}],
    'is_impossible': False,
    'question': 'What provides the following attribute?'}]},
 {'context': "\n>>> list(combined)\n['music', 'art', 'opera']. . . . maps A user updateable list of mappings.  The list is ordered from  first-searched to last-search",
  'qas': [{'id': 136,
    'answers': [{'answer_start': 69, 'text': 'first mapping'}],
    'is_impossib

In [ ]:
wandb server start

UsageError: unrecognized arguments: start


In [30]:
# Evaluate the model
result, texts = model.eval_model(test)

Running Evaluation:   0%|          | 0/2 [00:00<?, ?it/s]

In [31]:
# Make predictions with the model
to_predict = [
    {
        "context": "k”  and is short for “double-ended queue”).  Deques support thread-safe, memory  efficient appends and pops from either side of the deque with approxi",
        "qas": [
            {
                "question": "Deques support what kind of appends and pops?",
                "id": "126",
            }
        ],
    }
]

In [32]:
answers, probabilities = model.predict(to_predict)

print(answers)

add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 10538.45it/s]


Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

[{'id': '126', 'answer': []}]
